In [56]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve,auc
import os

from pathlib import Path
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [46]:
## Move file

# load  BAT  (stringent)
df_ = pd.read_csv('../VerifyAMPs/TransPi/truth_no_ambiguous/BAT_truth_set.stringent.csv', header=0)
from_path = '/mnt/vdb/Bat/pws/transpi/images/stringent/NonAMP'
to_path = '/mnt/vdb/Bat/pws/transpi/images/stringent/AMP'
check_ = []
for index, row in df_.iterrows():
    filename = row["prot_id"] + ".png" 
    Path(os.path.join(from_path, filename)).rename(os.path.join(to_path, filename))
    check_.append(os.path.join(to_path, filename))
# 235 
print(check_)


# load  DECockroach *(stringent)

df_ = pd.read_csv('../VerifyAMPs/TransPi/truth_no_ambiguous/DECockroach_truth_set.stringent.csv', header=0)

from_path = '/mnt/vdb/DECockroach/pws/transpi/images/stringent/NonAMP'
to_path = '/mnt/vdb/DECockroach/pws/transpi/images/stringent/AMP'
check_ = []
for index, row in df_.iterrows():
    filename = row["prot_id"] + ".png" 
    Path(os.path.join(from_path, filename)).rename(os.path.join(to_path, filename))
    check_.append(os.path.join(to_path, filename))
    # 138
print(len(check_))

# load  BAT  
df_ = pd.read_csv('../VerifyAMPs/TransPi/truth_no_ambiguous/BAT_truth_set.csv', header=0)
from_path = '/mnt/vdb/Bat/pws/transpi/images//NonAMP'
to_path = '/mnt/vdb/Bat/pws/transpi/images//AMP'
check_ = []


for index, row in df_.iterrows():
    try:
        filename = row["prot_id"] + ".png" 
        Path(os.path.join(from_path, filename)).rename(os.path.join(to_path, filename))
        check_.append(os.path.join(to_path, filename))
    except:
        print("Alrady placed:"+row["prot_id"])

print(len(check_)) #356

# load  DECockroach 

df_ = pd.read_csv('../VerifyAMPs/TransPi/truth_no_ambiguous/DECockroach_truth_set.csv', header=0)

from_path = '/mnt/vdb/DECockroach/pws/transpi/images/NonAMP'
to_path = '/mnt/vdb/DECockroach/pws/transpi/images/AMP'
check_ = []
for index, row in df_.iterrows():
    filename = row["prot_id"] + ".png" 
    Path(os.path.join(from_path, filename)).rename(os.path.join(to_path, filename))
    check_.append(os.path.join(to_path, filename))
print(len(check_)) #236

In [53]:
## load model 
deployed_path = "/mnt/vdb/thesis/CustomCNN.image.Adam.pssm.h5"
import tensorflow as tf

#with tf.device('/cpu:0'):
learner = load_model(deployed_path)

The order of the files that populate file_list, is the same order X_test appears in, by row.

So just match the indices to correlate filename with prediction.

X_test[0] ~ prediction[0] ~ file_list[0]

In [52]:
datagen = ImageDataGenerator(validation_split=0.2 )


truth_set = datagen.flow_from_directory(directory='../datasets/truthset/images/',
                                        class_mode='binary', batch_size=128,target_size=(200, 200),shuffle= False)

Bat_data = datagen.flow_from_directory(directory='/mnt/vdb/Bat/pws/transpi/images/normal',
                                      class_mode='binary', batch_size=128, target_size=(200, 200),shuffle= False)

Bat_stringent_data = datagen.flow_from_directory(directory='/mnt/vdb/Bat/pws/transpi/images/stringent',
                                        class_mode='binary', batch_size=128,target_size=(200, 200),
                                        shuffle= False)

DECockroach_data = datagen.flow_from_directory(directory='/mnt/vdb/DECockroach/pws/transpi/images/normal',
                                        class_mode='binary', batch_size=128,target_size=(200, 200),shuffle= False)

DECockroach_stringent_data = datagen.flow_from_directory(directory='/mnt/vdb/DECockroach/pws/transpi/images/stringent',
                                        class_mode='binary', batch_size=128,target_size=(200, 200),shuffle= False)
print(truth_set.class_indices)
print(Bat_data.class_indices)
print(Bat_stringent_data.class_indices)
print(DECockroach_data.class_indices)
print(DECockroach_stringent_data.class_indices)

Found 63 images belonging to 1 classes.
Found 10180 images belonging to 2 classes.
Found 10180 images belonging to 2 classes.
Found 15424 images belonging to 2 classes.
Found 15424 images belonging to 2 classes.
{'AMP': 0}
{'AMP': 0, 'NonAMP': 1}
{'AMP': 0, 'NonAMP': 1}
{'AMP': 0, 'NonAMP': 1}
{'AMP': 0, 'NonAMP': 1}


In [23]:
## test_truth_data
yhat = learner.predict_generator(test_truth_data)
_y = np.where(yhat > 0.5, 1, 0)
len(_y[_y ==1])

11

In [24]:
index = 0
image, label = test_truth_data._get_batches_of_transformed_samples(np.array([index]))
image_name = test_truth_data.filenames[index]

In [27]:
image_name

'AMP/Attacin-like_g1.png'

# New Groud Truthset

In [126]:
truth_set.filenames.pop(-1)

'AMP/.ipynb_checkpoints/Defensin_g14-checkpoint.png'

In [127]:
y_test = truth_set.classes
y_probas = learner.predict_generator(generator=truth_set)
y_pred = np.where(y_probas > 0.5, 1, 0)

In [128]:
len(y_pred[y_pred == 1]) # 11/62

11

# BAT

In [105]:
y_test = Bat_data.classes
y_probas = learner.predict_generator(generator=Bat_data)
y_pred = np.where(y_probas > 0.5, 1, 0)
print(classification_report(y_test, y_pred)) 

              precision    recall  f1-score   support

           0       0.04      0.14      0.06       356
           1       0.97      0.88      0.92      9824

    accuracy                           0.86     10180
   macro avg       0.50      0.51      0.49     10180
weighted avg       0.93      0.86      0.89     10180



In [106]:
AMP_list = y_pred[0:356]
len(AMP_list[AMP_list ==0])

49

In [107]:
nonAMP_list = y_pred[356:]
len(nonAMP_list[nonAMP_list ==1])

8663

### stringent

In [154]:
y_test = Bat_stringent_data.classes
y_probas = learner.predict_generator(generator=Bat_stringent_data)
y_pred = np.where(y_probas > 0.5, 1, 0)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.03      0.16      0.05       235
           1       0.98      0.88      0.93      9945

    accuracy                           0.87     10180
   macro avg       0.50      0.52      0.49     10180
weighted avg       0.96      0.87      0.91     10180



In [155]:
AMP_list = y_pred[0:235]
len(AMP_list[AMP_list ==0])

37

In [156]:
nonAMP_list = y_pred[235:]
len(nonAMP_list[nonAMP_list ==1])

8772

In [157]:
# Consensus

with open('../VerifyAMPs/TransPi/consensus/bat.final.truth') as f:
    bat_lines = [line.rstrip() for line in f]
# get index  inside list 
_truth_index = []
for i in bat_lines:
    search = "AMP/"+ i +".png"
    index = Bat_stringent_data.filenames.index(search)
    _truth_index.append(index)
_truth_index

[4, 8, 20, 61, 66, 113, 164, 175, 200, 201, 208, 215, 229]

In [158]:
y_pred[[4, 8, 20, 61, 66, 113, 164, 175, 200, 201, 208, 215, 229]]

array([[1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1]])

# DECockroach

In [111]:
y_test = DECockroach_data.classes
y_probas = learner.predict_generator(generator=DECockroach_data)
y_pred = np.where(y_probas > 0.5, 1, 0)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.02      0.11      0.03       236
           1       0.98      0.90      0.94     15188

    accuracy                           0.88     15424
   macro avg       0.50      0.50      0.48     15424
weighted avg       0.97      0.88      0.92     15424



In [112]:
AMP_list = y_pred[0:236]
len(AMP_list[AMP_list ==0])

25

In [113]:
nonAMP_list = y_pred[236:]
len(nonAMP_list[nonAMP_list ==1])

13619

### stringent

In [159]:
y_test = DECockroach_stringent_data.classes
y_probas = learner.predict_generator(generator=DECockroach_stringent_data)
y_pred = np.where(y_probas > 0.5, 1, 0)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.01      0.11      0.02       138
           1       0.99      0.90      0.94     15286

    accuracy                           0.89     15424
   macro avg       0.50      0.50      0.48     15424
weighted avg       0.98      0.89      0.93     15424



In [160]:
AMP_list = y_pred[0:138]
len(AMP_list[AMP_list ==0])

15

In [161]:
nonAMP_list = y_pred[138:]
len(nonAMP_list[nonAMP_list ==1])

13707

In [168]:
with open('../VerifyAMPs/TransPi/consensus/DECockroach.final') as f:
    de_lines = [line.rstrip() for line in f]
# get index  inside list 
_truth_index = []
for i in de_lines:
    search = "AMP/"+ i +".png"
    index = DECockroach_stringent_data.filenames.index(search)
    _truth_index.append(index)
_truth_index

[5, 14, 15, 16, 18, 32, 34, 47, 53, 65, 74, 78, 81, 82, 86, 104, 111, 117, 129]

In [169]:
y_pred[[5, 14, 15, 16, 18, 32, 34, 47, 53, 65, 74, 78, 81, 82, 86, 104, 111, 117, 129]]

array([[1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0]])